<a href="https://colab.research.google.com/github/iamslash/examplesofml/blob/master/kaggle/word2vec-nlp-tutorial/word2vec_nlp_tutoria_keras.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# !pip install kaggle
# from google.colab import files
# files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"iamslash","key":"3d11ff867c1693acf6519b33cd25290a"}'}

In [28]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import os
# print(os.listdir("../input"))

from bs4 import BeautifulSoup

import warnings
warnings.filterwarnings('ignore')

import re
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
g_stopwards_eng = set(stopwords.words("english"))

# make a clean text
def refine_text(s):
    s = s.lower()
    s = re.sub("[^a-z]", " ", s)
    l = [word for word in s.split() if word not in g_stopwards_eng]
    s = " ".join(l)
    return s

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

from keras.models import Sequential
from keras.layers import Embedding, LSTM, Dense, GlobalMaxPool1D, Dropout

cfg_num_words  = 6000
cfg_maxlen    = 2000
cfg_embed_size = 128
cfg_batch_size = 32
cfg_epochs = 2
cfg_validation_split = 0.01

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Using TensorFlow backend.


In [0]:
df_tr = pd.read_csv('../input/labeledTrainData.tsv', delimiter='\t', quoting=3)
df_te = pd.read_csv('../input/testData.tsv', delimiter='\t', quoting=3)


In [0]:
# get series of X, Y
sr_X_tr = df_tr.review
sr_Y_tr = df_tr.sentiment

sr_X_te = df_te.review

In [0]:
# refine X_tr
tkn = Tokenizer(num_words=cfg_num_words)

sr_X_tr = sr_X_tr.map(refine_text)
tkn.fit_on_texts(sr_X_tr)
sr_X_tr = pd.Series(tkn.texts_to_sequences(sr_X_tr))
# cfg_len_seq = sr_X_tr.map(lambda l : len(l)).max()
sr_X_tr = pad_sequences(sr_X_tr, maxlen=cfg_maxlen)

sr_X_te = sr_X_te.map(refine_text)
sr_X_te = pd.Series(tkn.texts_to_sequences(sr_X_te))
sr_X_te = pad_sequences(sr_X_te, maxlen=cfg_maxlen)

In [0]:
with tf.device('/gpu:0'):
  model = Sequential()
  model.add(Embedding(cfg_num_words, cfg_embed_size))
  model.add(LSTM(32, return_sequences=True))
  model.add(GlobalMaxPool1D())
  model.add(Dense(16, activation='relu'))
  model.add(Dropout(0.2))
  model.add(Dense(1, activation='sigmoid'))
  model.summary()
  model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
  model.fit(x=sr_X_tr, y=sr_Y_tr, batch_size=cfg_batch_size, epochs=cfg_epochs, validation_split=cfg_validation_split)    


(25000, 2000)